## Random IP Geolocation Visualizer

In [ ]:
import requests
import math
from ipwhois import IPWhois
from random import randint
from datetime import datetime
from ipyleaflet import Map, Marker
from IPython.display import Image

headers = {
    'accept': 'application/json',
    'content-type': 'application/json'
}

NASA_KEY = ''

Random IP generator - returns a string in the form of `xxx.xxx.xxx.xxx`, where `xxx` is an integer in range [0, 255].

In [ ]:
def random_IP():
    return '.'.join([str(randint(0,255)) for i in range(4)])

Generate a random IP address and ask [freegeoip.app](https://freegeoip.app/) for its Geolocation and other information. _Limited to 15k queries per hour._

In [ ]:
IP = random_IP()

url = 'https://freegeoip.app/json/' + IP
response = requests.request('GET', url, headers=headers)
lat = response.json()['latitude']
lon = response.json()['longitude']

print(IP, response.json()['country_name'], response.json()['region_name'], response.json()['city'], lat, lon, sep=', ')

Display the location (by latitude and longitude) on a map.

In [ ]:
%matplotlib inline

center = float(response.json()['latitude']), float(response.json()['longitude'])
m = Map(center=center, zoom=15)
m.add_layer(Marker(location=center, draggable=False));

display(m)

## Weather Data
_Using coordinates from the randomly generated IP._

Simple 7-day forecast, including weather conditions and max/min temperatures.

In [ ]:
mode = 'civillight'
meteo_url = f'http://www.7timer.info/bin/api.pl?lon={lon}&lat={lat}&product={mode}&output=json'
meteo_response = requests.request('GET', meteo_url, headers=headers)

In [ ]:
print('\nDate         Max    Min    Condition\n-----------------------------------')
for day in meteo_response.json()['dataseries']:
    temp_range = str(day['temp2m']['max']).rjust(2, ' ') + "°C   " + str(day['temp2m']['min']).rjust(2, ' ') + "°C"
    date_stamp = str(day['date'])[6:] + '.' + str(day['date'])[4:6] + '.' + str(day['date'])[:4]
    print(date_stamp, ' ', temp_range, ' ', day['weather'])

### WHOIS Information

Gather information about the randomly generated IP using a WHOIS lookup.

In [ ]:
lookup = IPWhois(IP)
results = lookup.lookup_whois()
for u, v in results['nets'][0].items():
    print(u + ':', v)

### Landsat Imagery

Get a Landsat 8 image for the closest available date and location. Using NASA's Earth [API](https://api.nasa.gov/).

_More about [Landsat 8](https://earthobservatory.nasa.gov/features/LandsatBigData)._

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
landsat_url = f'https://api.nasa.gov/planetary/earth/imagery?lon={lon}&lat={lat}&date={today}&api_key={NASA_KEY}'
landsat_response = requests.request('GET', landsat_url, headers=headers)

In [ ]:
Image(data=landsat_response.content, width=400)